In [8]:
import pandas as pd
from pyntcloud import PyntCloud
import numpy as np
import pdal
import shapely.wkt
from numpy.lib import recfunctions as rfn


In [9]:
def get_points(ept_path, bounds, wkt):
    
    READ_PIPELINE = """
                    {{
                        "pipeline": [
                            {{
                                "type": "readers.ept",
                                "filename": "{path}",
                                "bounds": "{bounds}"
                            }},
                            {{
                                "type":"filters.crop",
                                "polygon":"{wkt}"
                            }}
                        ]
                    }}
                    """
    
    pipeline = pdal.Pipeline(READ_PIPELINE.format(
        path=ept_path,
        bounds=bounds,
        wkt=wkt)
    )

    pipeline.validate()
    pipeline.execute()
    point_cloud = pipeline.arrays[0]

    return point_cloud



def write_to_laz(point_cloud, path):
    
    WRITE_PIPELINE = """
    {{
        "pipeline": [
            {{
                "type": "writers.las",
                "filename": "{path}",
                "extra_dims": "all"
            }}
        ]
    }}
    """
    pipeline = pdal.Pipeline(
        WRITE_PIPELINE.format(path=path),
        arrays=[point_cloud]
    )
    pipeline.validate()
    pipeline.execute()


In [10]:
test_area = [117813.6,487266.1,117942.1,487370.7]
xmin,ymin,xmax,ymax = test_area

path_a10_2018 = '/var/data/rws/data/2018/entwined/ept.json'
path_a10_2019 = '/var/data/rws/data/2019/amsterdam_entwined/ept.json'

# random area: 
# wkt = 'POLYGON((117813.6 487266.1,117813.6 487370.7,117942.1 487370.7,117942.1 487266.1,117813.6 487266.1))'

# small area: 
wkt = 'POLYGON((117813 487266,117813 487270,117815 487270,117815 487266,117813 487266))' # small! 

# portal:
# wkt = 'POLYGON((118032.6 488782.8,118032.6 488793.5,118078.8 488793.5,118078.8 488782.8,118032.6 488782.8))' # portal
xmin,ymin,xmax,ymax = shapely.wkt.loads(wkt).bounds

bounds = f'([{xmin}, {xmax}], [{ymin}, {ymax}])'
pc1 = get_points(path_a10_2018, bounds, wkt)
print(f'loaded pc 1')

loaded pc 1


In [11]:
pcdf = pd.DataFrame(np.array(pc1[['X','Y','Z','Red','Green','Blue']].tolist()), 
            columns=['x', 'y', 'z','red','green','blue'])



In [ ]:
pcdf = pd.DataFrame(np.array(pc1[['X','Y','Z']].tolist()), 
            columns=['x', 'y', 'z'])



In [12]:
import numpy as np
import pandas as pd
import matplotlib
pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from pyntcloud import PyntCloud
from sklearn.cluster import DBSCAN


min_size = 3
ransac_distance = 0.2
ransac_iterations = 1000
dbscan_distance = 1
dbscan_min_samples = 8


points = pd.DataFrame(
    np.array(pc1[['X','Y','Z']].tolist()), 
    columns=['x', 'y', 'z']
)

points_df = pd.DataFrame(np.array(pc1[['X','Y','Z','Red','Green','Blue']].tolist()), 
            columns=['x', 'y', 'z','red','green','blue'])

# points = np.array((x, y, z)).T
# points=pd.DataFrame(points,columns=['x','y','z'])
roof = PyntCloud(points)
#plpy.warning('Roofpoints in: ',len(roof.points))

eps1 = 1.2
min_size1 = 8
eps2 = 0.8
min_size2 = 6


In [22]:
ins, outs = find_plane(points_df)

In [14]:

print(points_df.shape, 
      np.array(ins['is_plane'].shape))

print(points_df)

(210, 7) [210]
              x           y      z      red    green     blue  is_plane
0    117813.889  487269.025  4.504  33791.0  25855.0  20479.0         1
1    117813.492  487266.317  4.491  32511.0  24063.0  19967.0         1
2    117814.257  487266.111  4.548  32767.0  25343.0  21503.0         1
3    117813.550  487267.833  4.486  36095.0  26623.0  22783.0         1
4    117814.275  487266.978  4.489  35327.0  24575.0  20991.0         1
5    117813.348  487268.739  4.461  37375.0  26367.0  22271.0         1
6    117814.348  487268.655  4.492  37119.0  26111.0  22015.0         1
7    117813.954  487269.262  4.477  35583.0  24319.0  20223.0         1
8    117814.969  487268.586  4.515  36607.0  25599.0  22271.0         1
9    117813.672  487269.399  4.476  34559.0  24063.0  20735.0         1
10   117813.483  487267.144  4.450  34303.0  22015.0  18943.0         1
11   117814.185  487267.697  4.449  34815.0  23807.0  19711.0         1
12   117813.030  487266.521  4.490  34559.0  2457

In [21]:
# np.ndarray(ins)
# points
print(points_df.shape, 
      np.array(ins['is_plane'].shape)
     )

out_array = rfn.append_fields(points_df, 
                              'cid', 
                              np.array(ins['is_plane']),
                              dtypes= '?'
                             )

print(out_array[0])

# write_to_laz(out_array, '/var/data/rws/data/output/tst_c1.laz')

(210, 7) [210]
('x', 'y', 'z', 'red', 'green', 'blue', 'is_plane', True)


In [52]:
def find_clusters(points, eps, min_samples):
    df = points[["x", "y","z"]]
    db = DBSCAN(eps=eps, min_samples=min_samples)
    dbsc = db.fit(df)
    labels = dbsc.labels_
    core_samples = np.zeros_like(labels, dtype = bool)
    core_samples[dbsc.core_sample_indices_] = True
    points["coresample"]= core_samples
    points["cluster"] = dbsc.labels_
    return points

def find_plane(points):
    pointcloud = PyntCloud(points)
    pointcloud.add_scalar_field("plane_fit",
                                max_dist=ransac_distance, 
                                max_iterations=ransac_iterations, 
                                n_inliers_to_stop=None)
    
    inliers = pointcloud.points[pointcloud.points.is_plane == 1]
    outliers = pointcloud.points[pointcloud.points.is_plane == 0]
    
    return inliers, outliers

global planes
planes=pd.DataFrame(columns=['x','y','z','cid'])
def recurse(roof,cid):
    global planes
    #plpy.warning('Iter ',cid,' with ',len(roof.points),'points')
    rest=pd.DataFrame(columns=['x','y','z'])
    
    points = find_clusters(roof.points,eps1,min_size1) #split clusters and remove garbage
    clusters = points[(points.cluster >= 0)]
    _ = points[(points.cluster < 0)]
    counts = np.bincount(clusters['cluster'])
    #max = np.argmax(counts)
    #maxcluster = clusters[clusters.cluster == max]
    #restcluster = clusters[clusters.cluster != max]
    
    if len(clusters) > 0:
      planeinliers, planeoutliers = find_plane(clusters) #find best plane, keep remainder
      
      points = find_clusters(planeinliers,eps2,min_size2) #split plane in possible clusters, keep remainder
      clusterinliers = points[(points.cluster >= 0) & (points.coresample == True)]
      clusteroutliers = points[(points.cluster < 0) | ((points.cluster < 0) & (points.coresample == False))]

      rest = pd.concat([rest,planeoutliers,clusteroutliers]) #keep all remainders for further use
      if len(clusterinliers) > min_size:
          #print(' ',clusterinliers['cluster'].max() + 1, 'clusters will be added')
          for name, group in clusterinliers.groupby('cluster'): 
              cid = cid + 1
              group['cid'] = cid #assign a value to the newly found cluster
              planes = pd.concat([planes,group]) #add to the plane stack
    else:
      return
    #plpy.warning('Number of rest points: ', len(rest))
    if len(rest) <= min_size or cid > 100 or len(rest) == len(roof.points):
      return
    else:
      recurse(PyntCloud(rest),cid)

In [49]:
print(points.shape)
print(planes['cid'].shape)
print(pc1.shape)

(210, 3)
(209,)
(210,)


In [65]:
roof.points["cid"] = -1
recurse(roof,0)
# points = np.array((planes['x'], planes['y'],planes['z'],planes['cid'])).T


# write_to_laz(out_array, '/var/data/rws/data/output/tst_c1.laz')

(0, 10)


TypeError: Cannot change data-type for object array.

In [44]:
planes

,x,y,z,cid


In [19]:
print(np.unique(planes['cid'], return_counts =True))
c1 = points[points[:,3] == 1][:,0:3]
c2 = points[points[:,3] == 2][:,0:3]


(array([1], dtype=object), array([209]))


In [20]:
print(c1.shape)
write_to_laz(c1, '/var/data/rws/data/output/tst_portal_c1.laz')
print(c2.shape)
write_to_laz(c2, '/var/data/rws/data/output/tst_portal_c2.laz')
print('done')

(209, 3)


RuntimeError: Array without fields must have 3 dimensions.

In [26]:
pyntcloud.scalar_fields.PlaneFit(*, pyntcloud, max_dist=0.0001,
max_iterations=100, n_inliers_to_stop=None)

SyntaxError: invalid syntax (<ipython-input-26-89eca175caa0>, line 1)

In [21]:
# print(c2.shape)
write_to_laz(pc1, '/var/data/rws/data/output/tst_portal_c2.laz')

In [25]:
type(pc1)

numpy.ndarray